In [81]:
#pip install mysql-connector-python

In [374]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [375]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [376]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [377]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [378]:
c = create_db_connection("giniewicz.it","team06","te@m0g","team06")

MySQL Database connection successful


In [379]:
#suma zarobionych pieniędzy ze sprzedaży gier w styczniu 2022
q1 = """
SELECT sum(game_purchase.price)
FROM game_purchase
WHERE DATE(game_purchase.purchase_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

#suma zarobionych pieniędzy z wypożyczeń gier w styczniu 2022
q2 = """
SELECT sum(game_rental.price)
FROM game_rental
WHERE DATE(game_rental.rental_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

# suma zarobków pracowników
q3 = """
SELECT sum(salary)
FROM employees
"""

#pracownicy
q4 = """
SELECT *
FROM employees
"""

#za ile pieniędzy w danym miesiącu pracownik sprzedał gier
q5 = """
SELECT sum(price) as t, employee_id, month(purchase_date) as m, year(purchase_date) as y, 
concat(employee_id,month(purchase_date),year(purchase_date)) as id
FROM game_purchase
GROUP BY y, m, employee_id
"""

#za ile pieniędzy w danym miesiącu pracownik wypożyczył gier
q6 = """
SELECT sum(game_rental.price) as t, game_rental.employee_id, month(game_rental.rental_date) as m, year(game_rental.rental_date) as y, 
concat(game_rental.employee_id,month(game_rental.rental_date),year(game_rental.rental_date)) as id,
concat(employees.first_name," ",employees.last_name) as names
FROM game_rental INNER JOIN employees on game_rental.employee_id = employees.employee_id
GROUP BY y, m, game_rental.employee_id
"""

#przychody ze sprzedaży
q7 = """
SELECT sum(game_purchase.price), shop_inventory.game_title
FROM game_purchase INNER JOIN shop_inventory ON game_purchase.game_id = shop_inventory.game_id
GROUP BY game_purchase.game_id
ORDER BY sum(game_purchase.price) DESC
LIMIT 5;
"""

#przychody z wypożyczeń
q8 = """
SELECT sum(game_rental.price), shop_inventory.game_title
FROM game_rental INNER JOIN shop_inventory ON game_rental.game_id = shop_inventory.game_id
GROUP BY game_rental.game_id
ORDER BY sum(game_rental.price) DESC
LIMIT 5;
"""



In [380]:
r1 = float(read_query(c,q1)[0][0])
r2 = float(read_query(c,q2)[0][0])
r3 = float(read_query(c,q3)[0][0])
r4 = read_query(c,q4)
r5 = read_query(c,q5)
r6 = read_query(c,q6)
r7 = read_query(c,q7)
r8 = read_query(c,q8)

In [381]:
df1 = pd.DataFrame({
    "purchase_income": [r1],
    "rental_income": [r2],
    "employees_cost": [r3]})
df4 = pd.DataFrame(r4, columns =['employee_id', 'first_name', 'last_name', 'salary'])
df5 = pd.DataFrame(r5, columns =['sold_games_price', 'employee_id', 'month', 'year',"id"])
df6 = pd.DataFrame(r6, columns =['rented_games_price', 'employee_id_r', 'month_r', 'year_r',"id","name"])
df6_1 = df6.join(df5.set_index('id'), on='id')
df7 = pd.DataFrame(r7, columns =["purchase_income","game_title"])
df8 = pd.DataFrame(r8, columns =["rental_income","game_title"])

In [382]:
# zysk za styczeń 2022
s = r1+r2-r3

In [383]:
# top pracownicy
df6_1["sold_games_price"] = df6_1["sold_games_price"].fillna(0)
df6_1["total"] =( df6_1["rented_games_price"] + df6_1["sold_games_price"]).astype(int)
max_value_idx = df6_1.groupby(['year_r', 'month_r'])['total'].idxmax()
max_value_employees = df6_1.loc[max_value_idx][["name","total","month_r","year_r"]].rename(columns={"name": "top_employee"})

In [384]:
html = f'''
<html>
    <head>
        <title>team06</title>
        <style>
            .highlight {{
                font-weight: bold;
                color: green;
            }}
        </style>
    </head>
    <body>
        <h1>Raport - analiza danych.</h1>
        <h2>1. Pracownicy miesiąca</h2>
        {max_value_employees.to_html()}
        
        <h2>2. Top 10 zawodników</h2>
    
        
        <h2>3. Przychody w zależności od sprzedaży i wypożyczeń</h2>
        <p>Największe przychody ze sprzedaży:</p>
        {df7.to_html()}
        <p>Największe przychody z wypożyczeń:</p>
        {df8.to_html()}
        
        <h2>4. Własne</h2>
        <h3>4.1 Zyski za styczeń 2022</h3>
        {df1.to_html()}
        <p>Zyski z działalności wypożyczalni gier w styczniu 2022,
        obliczone jako różnica między sumą z przychodów ze sprzedaży i wypożyczeń gier,
        a sumą miesięcznych pensji pracowników:</p>
        <p><span class="highlight">{s} zł</span></p>
        
        <h3>4.2 </h3>
         
    </body>
</html>
    '''

with open('html_report.html', 'w') as f:
    f.write(html)